**LCEL 임의의 체인에 메모리 추가**

In [17]:
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda , RunnablePassthrough
from langchain_openai import ChatOpenAI


In [19]:
model = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature= 0,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful chatbot')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])


    ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful chatbot')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [20]:
# 대화내용을 저장할 메모리인 ConversationBufferMemory를 생성하고 return_messages=True로 생성한 인스턴스가 메시지를 반환하도록 한다. 
# memory_key : chain의 prompt 안에 대입될  key, 변경해서 사용

# 대화 버퍼 메모리를 생성하고 메시지 반환 기능 활성화

memory = ConversationBufferMemory(
    return_messages=True, memory_key='chat_history'
)

In [21]:
memory.load_memory_variables({})

{'chat_history': []}

In [22]:
# RunnablePassthrough.assign을 사용해 chat_history에 memory.load_memory_variables 함수 할당

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter('chat_history')
)

runnable

RunnableAssign(mapper={
  chat_history: RunnableLambda(load_memory_variables)
                | RunnableLambda(itemgetter('chat_history'))
})

In [23]:
# runnable 에 첫 번째 대화

runnable.invoke({"input" : "ㅎ2"})

{'input': 'ㅎ2', 'chat_history': []}

In [24]:
chain = runnable | prompt | model

chain

RunnableAssign(mapper={
  chat_history: RunnableLambda(load_memory_variables)
                | RunnableLambda(itemgetter('chat_history'))
})
| ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful chatbot')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11d6da920>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11d58beb0>, temperature=0.0, openai_a


      RunnableAssign(mapper={
    chat_history: RunnableLambda(load_memory_variables)
                  | RunnableLambda(itemgetter('chat_history'))
    })
    | ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful chatbot')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
    | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11d588970>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11d589e10>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [25]:
response = chain.invoke({"input" : "하이용 저는 김뫄뫄입니다."})
response

AIMessage(content='안녕하세요! 궁금한 것이 있나요?', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 34, 'total_tokens': 52}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [26]:
memory.save_context(
    {"inputs": "하이용 저는 김뫄뫄입니다."},
    {"output": response.content}
)

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='하이용 저는 김뫄뫄입니다.'),
  AIMessage(content='안녕하세요! 궁금한 것이 있나요?')]}

In [29]:
response = chain.invoke({"input" : "나의 이름은?"})
print(response.content)

죄송합니다, 제가 이름을 알 수 있는 방법은 없습니다. 그래서 김뫄뫄라고 부르겠습니다. 어떻게 도와드릴까요?


In [28]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='하이용 저는 김뫄뫄입니다.'),
  AIMessage(content='안녕하세요! 궁금한 것이 있나요?')]}